In [12]:
# 필요한 라이브러리 import하기

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import optuna 
import lightgbm as lgb 
from sklearn.metrics import accuracy_score, roc_auc_score,precision_score, confusion_matrix,recall_score 
from sklearn.model_selection import cross_val_score, train_test_split
import os 

In [13]:
# train data와 test data 로드하기

train_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/wine_train_data.csv')
test_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/wine_test_data.csv')

In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1279 entries, 0 to 1278
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1279 non-null   float64
 1   volatile acidity      1279 non-null   float64
 2   citric acid           1279 non-null   float64
 3   residual sugar        1279 non-null   float64
 4   chlorides             1279 non-null   float64
 5   free sulfur dioxide   1279 non-null   float64
 6   total sulfur dioxide  1279 non-null   float64
 7   density               1279 non-null   float64
 8   pH                    1279 non-null   float64
 9   sulphates             1279 non-null   float64
 10  alcohol               1279 non-null   float64
 11  quality               1279 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 120.0 KB


In [15]:
train_X=train_data.drop('quality',axis=1)
train_y=train_data['quality']

new_test_data_X=test_data.drop('quality',axis=1)
new_test_data_y=test_data['quality']

In [16]:
train_y.value_counts()

0    684
1    595
Name: quality, dtype: int64

In [17]:
# class weight 적용하기 위해 y label의 비율을 구하기

ratio=train_y.value_counts()[0]/train_y.value_counts()[1]

## 베이지안 최적화해주기 - LightgbmClassifier

**1. 5-fold 교차검증 이용해서 Train data로 Validation set을 Auc가 최고였을 때의 하이퍼파라미터 구하기**

**2. Learning rate 0.01~0.1, max_depth 3~9, n_estimators 100~1000이었을 때에서 가장 최적의 하이퍼파라미터 구하기**

In [8]:
def objective(trial):
    # Define hyperparameters to optimize 
    params={
        'boosting_type':'gbdt',
        'objective':'binary',
        'metric':'binary_logloss',
        'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
        'max_depth':trial.suggest_int('max_depth',3,9),
        'n_estimators':trial.suggest_int("n_estimators",100,1000)
    }
    # Train and evaluate model 
    lgb_cv=lgb.LGBMClassifier(**params, random_state=42,scale_pos_weight=ratio)
    scores=cross_val_score(lgb_cv,train_X,train_y,cv=5,scoring='roc_auc')
    auc=scores.mean()
    return auc 

# Define study object and optimize 

study=optuna.create_study(direction='maximize',study_name='lgb_boost_opt',load_if_exists=True)
study.optimize(objective, n_trials=50)

# Print best hyperparameters and auc
print(f'Best hyperparameters: {study.best_params}')
print(f'Best AUC: {study.best_value:.4f}')

[I 2023-04-28 13:26:05,096] A new study created in memory with name: lgb_boost_opt
/tmp/ipykernel_827478/2141867322.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-28 13:26:08,520] Trial 0 finished with value: 0.862408939531158 and parameters: {'learning_rate': 0.043960208687925204, 'max_depth': 7, 'n_estimators': 626}. Best is trial 0 with value: 0.862408939531158.
/tmp/ipykernel_827478/2141867322.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-28 13:26:09,636] Trial 1 finished wi

Best hyperparameters: {'learning_rate': 0.03841283666283786, 'max_depth': 9, 'n_estimators': 682}
Best AUC: 0.8680


In [18]:
lgb=lgb.LGBMClassifier(learning_rate= 0.03841283666283786,max_depth=9,n_estimators=682,scale_pos_weight=ratio,random_state=42)

In [19]:
# 모델 학습하기

lgb.fit(train_X,train_y)

LGBMClassifier(learning_rate=0.03841283666283786, max_depth=9, n_estimators=682,
               random_state=42, scale_pos_weight=1.149579831932773)

In [20]:
# 모델의 성능(AUC)을 평가하기 위해 예측된 클래스가 불량일 확률 구하기

prob=lgb.predict_proba(new_test_data_X)[:,1]

In [21]:
# 모델 성능 평가함수 만들기

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_prob(y_test, probability):
  pred=np.where(probability > 0.5,1,0)
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred) 
  recall=recall_score(y_test,pred) 
  # F1 스코어 추가 
  f1=f1_score(y_test,pred,average='macro')
  Roc_score=roc_auc_score(y_test,probability)
  print('임계값: ', 0.5) 
  print('오차행렬')
  print(confusion) 
  # f1 score print 추가 
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4: .4f}'.format(accuracy,precision,recall,f1,Roc_score))

In [22]:
# 모델 평가하기

get_clf_prob(new_test_data_y,prob)

임계값:  0.5
오차행렬
[[148  23]
 [ 24 125]]
정확도: 0.8531, 정밀도: 0.8446, 재현율: 0.8389, F1:0.8524, AUC: 0.9070
